In [182]:
from sentence_transformers import SentenceTransformer
import requests 
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import json
from elasticsearch import Elasticsearch

In [183]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [184]:
filtered_documents = filter(lambda documents: documents['course'] == 'machine-learning-zoomcamp', documents)

filtered_documents = list(filtered_documents)

print(len(filtered_documents))

375


In [185]:
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [186]:
len(model.encode("This is a simple sentence"))

768

In [187]:
operations = []
for doc in filtered_documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [188]:
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'd436f79b3f72', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ERyZxAUUTwqGK5N8HxZtjA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [189]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [190]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [191]:
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [192]:
search_term = "I just discovered the course. Can I still join it?"
vector_search_term = model.encode(search_term)

In [193]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 10,
    "num_candidates": 10000, 
}

In [194]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

# Caq2tZABx8A_rsVbQeq3

[{'_index': 'course-questions',
  '_id': 'Caq2tZABx8A_rsVbQeq3',
  '_score': 0.73748624,
  '_source': {'question': 'When does the next iteration start?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well).'}},
 {'_index': 'course-questions',
  '_id': '_6q2tZABx8A_rsVbQel_',
  '_score': 0.7361103,
  '_source': {'question': "I filled the form, but haven't received a confirmation email. Is it normal?",
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': "The process is automated now, so you should receive the email eventually. If you haven’t, check your promotions tab in Gmail as well as spam.\nIf you unsubscribed from our 

In [198]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'

ground_truth_relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{ground_truth_relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [203]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    relevance = [d['_id'] == doc_id for d in res["hits"]["hits"]]
    relevance_total.append(relevance)

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

relevance_total

print(hit_rate(relevance_total))

  0%|          | 0/1830 [00:00<?, ?it/s]

0.0
